## [1] 1차 전처리
+ 날짜 바꾸기 : 2022 -> 2021 잘못된 것 수정
+ 2021-09-07 ~ 2022-09-06 모든 날짜 column으로 넣기

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/대형마트/농산물유통정보

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/대형마트/농산물유통정보


### 저장할 df 만들기

In [44]:
### 저장할 df 만들기
import datetime
import pandas as pd

start = datetime.date(2021, 9, 7)
end = datetime.date(2022,9,6)

### columns
columns = ['goods_name'] +[ (start + datetime.timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end-start).days+1)    ]

### df만들기
pdf = pd.DataFrame(columns=columns)
print(pdf.shape)
pdf

(0, 366)


,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,2021-09-14,2021-09-15,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06


### 2022 -> 2021 로 날짜 바꾸기

In [45]:
### 날짜 바꾸기
import numpy as np
# 파라미터 예시
split_date = datetime.date(2022, 9, 7) # 기준이 되는 날짜로, 우리는 2021-09-07 - 2022-09-06까지 데이터를 조회했음
# tmp는 dataframe

# 함수
def change_date_2022_to_2021(split_date, tmp): # -> List[datetime.date]

    start = 1
    while type(tmp.loc[0,start]) != datetime.datetime:
        start += 1
    
    period_lst = list(tmp.loc[0,start:])


    for j in range(len(period_lst)):
        date = period_lst[j].date()

        # 2021년 경우
        if date >= split_date:
            date = date - relativedelta(years=1)  # 1년 빼기
        period_lst[j] = date
    
    return period_lst   # List[datetime.date] 형식으로 output

### 빠진 date 추가하기

In [47]:
### date 추가하기 
import datetime  # + timedelta(days=1) 이용
import numpy as np


# 파라미터 예시
start = datetime.date(2021, 9, 7)
end = datetime.date(2022, 9, 6)
# period_lst = list(tmp.loc[0,2:])
# mean_price_lst = list(tmp.loc[1,2:])

# 함수
def adding_date(start, end, period_lst, mean_price_lst): # -> List[np.nan | float]
    ### 앞쪽 결측치 넣어주기
    add_front_period = []
    add_front_mean_price = []
    cur = start
    while period_lst[0] > cur:
        add_front_period.append(cur)
        cur += datetime.timedelta(days=1)
    add_front_mean_price = [np.nan]*len(add_front_period)

    ### 뒤쪽 결측치 넣어주기
    add_back_period = []
    add_back_mean_price = []
    cur = end
    while period_lst[-1] < cur:
        add_back_period.append(cur)
        cur -= datetime.timedelta(days=1)
    add_back_period = add_back_period[::-1]  # 순서 뒤집어 주기
    add_back_mean_price = [np.nan]*len(add_back_period)

    ### 중간결측치 넣어주기
    period_lst += [-1]
    mean_price_lst += [-1]

    i = 0
    while period_lst[i+1] != -1:
        if (period_lst[i+1] - period_lst[i]).days != 1:
            period_lst.insert(i+1,period_lst[i] + datetime.timedelta(days=1) )  # 다음날 추가   [1,4,5,6,7]  [1,2,4,5,6,7]
            mean_price_lst.insert(i+1, np.nan)
        i += 1


    period_lst.pop()      # -1 없애기
    mean_price_lst.pop()  # -1 없애기



    ### 결측치 합치기
    period_lst = add_front_period + period_lst + add_back_period
    mean_price_lst = add_front_mean_price + mean_price_lst + add_back_mean_price
   



    return mean_price_lst

In [48]:
# xlsx 파일 모두 읽고, pdf 채우기
import pandas as pd
from dateutil.relativedelta import relativedelta
import datetime
from glob import glob
import numpy as np


paths = ['./과일류/', './식량작물/', './채소류/', './특용작물/']
for path in paths:
    for filename in glob(path+'*.xlsx'):
        # print(filename)
        goods_name = ''.join(filename.split('_')[3:]).split('.')[0]
        tmp = pd.read_excel(filename, header=None)
        
        # 2022 -> 2021로 날짜바꾸기
        split_date = datetime.date(2022, 9, 7) # 기준이 되는 날짜로, 우리는 2021-09-07 - 2022-09-06까지 데이터를 조회했음
        period_lst =  change_date_2022_to_2021(split_date, tmp)

        # 데이터 가져올 시작 열 설정
        start = 1
        while type(tmp.loc[0,start]) != datetime.datetime:
            start += 1

        mean_price_lst = list(tmp.loc[1,start:])

        # 빈 날짜 채워주기
        start = datetime.date(2021, 9, 7)
        end = datetime.date(2022, 9, 6)
        mean_price_lst = adding_date(start, end, period_lst, mean_price_lst)

        # 결과 df에 모아주기
        one_goods_as_row = [goods_name] + mean_price_lst
        # print(len(one_goods_as_row))
        pdf.loc[len(pdf)] = one_goods_as_row
        

In [49]:
pdf

,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,2021-09-14,2021-09-15,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,사과후지(중),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,감귤노지(s과),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,감귤시설(s과),"6,855","6,859","6,859","6,901",NaN,NaN,"6,890","6,892","6,855",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,건블루베리수입(중품),"3,220","3,220","3,200","3,200",NaN,NaN,"3,200","3,200","3,200",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,건포도수입(중품),"1,033","1,045","1,045","1,033",NaN,NaN,"1,033","1,033","1,033",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,단감단감(중품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,레몬수입(중품),"8,786","8,786","9,072","9,065",NaN,NaN,"9,065","9,065","9,065",...,NaN,"9,507","9,507","9,568","9,568","9,614",NaN,NaN,"9,652","9,652"
7,망고수입(상품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,바나나수입(상품),284,284,284,284,NaN,NaN,284,284,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
9,배신고(중),NaN,NaN,"30,830","25,405",NaN,NaN,"30,428","29,760","30,161",...,NaN,NaN,NaN,NaN,"23,230","22,780",NaN,NaN,"22,480","22,480"


### 저장하기 (1차 전처리)

In [50]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/대형마트/농산물유통정보

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/대형마트/농산물유통정보


In [51]:
pdf.to_excel('농산물유통정보_일데이터_대형마트_1년_상품_가격_1차전처리.xlsx',header=True, index=False)

## [2] 2차 전처리 - 상품명 넣기 & 품목넣기 

### (1) data 가져오기

In [3]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/대형마트/농산물유통정보

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/대형마트/농산물유통정보


In [52]:
import pandas as pd
pdf = pd.read_excel('농산물유통정보_일데이터_대형마트_1년_상품_가격_1차전처리.xlsx')
pdf

,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,2021-09-14,2021-09-15,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,사과후지(중),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,감귤노지(s과),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,감귤시설(s과),"6,855","6,859","6,859","6,901",NaN,NaN,"6,890","6,892","6,855",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,건블루베리수입(중품),"3,220","3,220","3,200","3,200",NaN,NaN,"3,200","3,200","3,200",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,건포도수입(중품),"1,033","1,045","1,045","1,033",NaN,NaN,"1,033","1,033","1,033",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,단감단감(중품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,레몬수입(중품),"8,786","8,786","9,072","9,065",NaN,NaN,"9,065","9,065","9,065",...,NaN,"9,507","9,507","9,568","9,568","9,614",NaN,NaN,"9,652","9,652"
7,망고수입(상품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,바나나수입(상품),284,284,284,284,NaN,NaN,284,284,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
9,배신고(중),NaN,NaN,"30,830","25,405",NaN,NaN,"30,428","29,760","30,161",...,NaN,NaN,NaN,NaN,"23,230","22,780",NaN,NaN,"22,480","22,480"


### (2) 품목명과 id dictionary 가져오기

In [53]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/월별데이터(10년)

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/월별데이터(10년)


In [ ]:
### 품목명과 id 가져오기
import pickle
with open('product_productId_dict_nong.pickle', 'rb') as fr:
    product_productId_dict_nong = pickle.load(fr)
product_productId_dict_nong

### (3) 품목과 품목id 넣기

In [ ]:
### column 생성
pdf.insert(0,'product_id', [-1]*len(pdf))
pdf.insert(1,'product', [-1]*len(pdf))
pdf



In [36]:
### 참고
### 한글의 경우, 자모가 분리되는 경우와 결합되는 경우로 나뉘어지기 때문에 아래와 같은 이상한 일이 벌어짐
str1 = '건블루베리수입(중품)'
str2 = '건블루베리수입(중품)'
print(len(str1))
print(len(str2))

## 그래서 자모를 결합하는 경우로 맞춰줘야 함!
import unicodedata

# 경우 1 : 한글 문자열을 자음 모음으로 분리하기
uni1 = unicodedata.normalize('NFD',str1)

# 경우 2 : 분리된 자음 모음을 결합하기
uni2 = unicodedata.normalize('NFC',str2)

print('변경후')
print(len(uni1))
print(len(uni2))

11
25
변경후
25
11


In [56]:
### product & product_id 넣어주기

import unicodedata
for i in range(len(pdf)):
    # goods 와 비교해서 품목 넣어주기
    for key in product_productId_dict_nong:
        if key in unicodedata.normalize('NFC',pdf.loc[i,'goods_name']):
            pdf.loc[i,'product'] = key
            pdf.loc[i,'product_id'] = product_productId_dict_nong[key]

### (4) 전처리 pdf의 goods_name 가져와서, (3)의 결과물에 품목 제대로 넣어주기

In [71]:
products = [
'사과',
'감귤',
'감귤',
'건블루베리',
'건포도',
'단감',
'레몬',
'망고',
'바나나',
'배',
'복숭아',
'-1',
'오렌지',
'참다래',
'참다래',
'체리',
'파인애플',
'포도',
'포도',
'고구마',
'녹두',
'쌀',
'찹쌀',
'콩',
'팥',
'감자',
'고춧가루',
'깐마늘',
'깻잎',
'무',
'딸기',
'멜론',
'무',
'방울토마토',
'배추',
'붉은고추',
'상추',
'생강',
'수박',
'시금치',
'배추',
'파',
'배추',
'무',
'오이',
'참외',
'토마토',
'파',
'파프리카',
'풋고추',
'피망',
'호박',
'느타리버섯',
'땅콩',
'땅콩',
'새송이버섯',
'아몬드',
'참깨',
'팽이버섯',
'호두']

In [74]:
goods_name=\
['사과후지(중)',
'감귤노지(s과)',
'감귤시설(s과)',
'건블루베리수입(중품)',
'건포도수입(중품)',
'단감단감(중품)',
'레몬수입(중품)',
'망고수입(상품)',
'바나나수입(상품)',
'배신고(중)',
'복숭아백도(상)',
'아보카도수입(상품)',
'오렌지네이블미국(상품)',
'참다래국산(상품)',
'참다래그린뉴질랜드(상품)',
'체리수입(중품)',
'파인애플수입(중품)',
'포도샤인머스켓(중)',
'포도캠벨얼리(중)',
'고구마밤',
'녹두국산',
'쌀일반계',
'찹쌀일반계',
'콩흰 콩(국산)',
'팥붉은 팥(국산)',
'감자수미',
'고춧가루국산',
'깐마늘깐마늘',
'깻잎깻잎',
'당근무세척',
'딸기딸기',
'멜론멜론',
'무전체',
'방울토마토방울토마토',
'배추전체',
'붉은고추붉은고추',
'상추적',
'생강국산',
'수박수박',
'시금치시금치',
'양배추양배추(중급)',
'양파전체',
'얼갈이배추얼갈이배추',
'열무열무',
'오이가시계통',
'참외참외',
'토마토토마토',
'파대파',
'파프리카파프리카',
'풋고추풋고추',
'피망청',
'호박애호박',
'느타리버섯애느타리버섯',
'땅콩국산',
'땅콩수입(중품)',
'새송이버섯새송이버섯',
'아몬드수입(중품)',
'참깨백색(국산)',
'팽이버섯팽이버섯',
'호두수입(중품)']

In [75]:
len(products), len(goods_name)

(60, 60)

In [84]:
goods_product_dict_only_for_preprocess = {goods : product for goods, product in zip(goods_name,products)}

In [ ]:
goods_product_dict_only_for_preprocess

In [96]:
# 삭제할 데이터 선정
goods_product_dict_only_for_preprocess['포도샤인머스켓(중)'] = '-1'


# 수정할 데이터 선정
goods_product_dict_only_for_preprocess['당근무세척'] = '당근'
goods_product_dict_only_for_preprocess['양배추양배추(중급)'] = '양배추'
goods_product_dict_only_for_preprocess['양파전체'] = '양파'
goods_product_dict_only_for_preprocess['얼갈이배추얼갈이배추'] = '얼갈이배추'
goods_product_dict_only_for_preprocess['열무열무'] = '열무'


In [97]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/대형마트/농산물유통정보

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/대형마트/농산물유통정보


In [ ]:
# pickle로 dictionary 저장
import pickle
with open('goods_product_dict_only_for_preprocess.pickle','wb') as fw:
    pickle.dump(goods_product_dict_only_for_preprocess, fw)

In [98]:
# pickle로 dictionary 로드
import pickle
with open('goods_product_dict_only_for_preprocess.pickle','rb') as fr:
    goods_product_dict_only_for_preprocess = pickle.load(fr)

In [109]:
### product_goods_dict_only_for_preprocess 이용해서, pdf의 product 변경
for i in range(len(pdf)-1, -1, -1):
    goods_name = pdf.loc[i,'goods_name']


    # 삭제 or 변경
    if goods_product_dict_only_for_preprocess[goods_name] == '-1':
        pdf.drop(i, inplace=True) # i번째 행 삭제
    else:
        pdf.loc[i,'product'] = goods_product_dict_only_for_preprocess[goods_name]
        pdf.loc[i,'product_id'] = product_productId_dict_nong[goods_product_dict_only_for_preprocess[goods_name]]  # 품목 id 제대로 넣어주기

In [110]:
pdf.reset_index(drop=True, inplace=True)
pdf

,product_id,product,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,1038,사과,사과후지(중),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1054,감귤,감귤노지(s과),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1054,감귤,감귤시설(s과),"6,855","6,859","6,859","6,901",NaN,NaN,"6,890",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1050,건블루베리,건블루베리수입(중품),"3,220","3,220","3,200","3,200",NaN,NaN,"3,200",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1049,건포도,건포도수입(중품),"1,033","1,045","1,045","1,033",NaN,NaN,"1,033",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1042,단감,단감단감(중품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1047,레몬,레몬수입(중품),"8,786","8,786","9,072","9,065",NaN,NaN,"9,065",...,NaN,"9,507","9,507","9,568","9,568","9,614",NaN,NaN,"9,652","9,652"
7,1051,망고,망고수입(상품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1043,바나나,바나나수입(상품),284,284,284,284,NaN,NaN,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
9,1039,배,배신고(중),NaN,NaN,"30,830","25,405",NaN,NaN,"30,428",...,NaN,NaN,NaN,NaN,"23,230","22,780",NaN,NaN,"22,480","22,480"


In [59]:
len(product_productId_dict_nong)

55

### (5) 데이터 숫자로 바꾸기

In [133]:
type(pdf.iloc[57,3])

str

In [134]:
import numpy as np

for i in range(len(pdf)):
    for j in range(3,pdf.shape[1] ):
        p = pdf.iloc[i,j]
        if type(p)==str:
            p = int(p.replace(',',''))
        pdf.iloc[i,j] = p

In [135]:
pdf

,product_id,product,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,1038,사과,사과후지(중),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1054,감귤,감귤노지(s과),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1054,감귤,감귤시설(s과),6855,6859,6859,6901,NaN,NaN,6890,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1050,건블루베리,건블루베리수입(중품),3220,3220,3200,3200,NaN,NaN,3200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1049,건포도,건포도수입(중품),1033,1045,1045,1033,NaN,NaN,1033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1042,단감,단감단감(중품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1047,레몬,레몬수입(중품),8786,8786,9072,9065,NaN,NaN,9065,...,NaN,9507,9507,9568,9568,9614,NaN,NaN,9652,9652
7,1051,망고,망고수입(상품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1043,바나나,바나나수입(상품),284,284,284,284,NaN,NaN,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
9,1039,배,배신고(중),NaN,NaN,30830,25405,NaN,NaN,30428,...,NaN,NaN,NaN,NaN,23230,22780,NaN,NaN,22480,22480


In [123]:
pdf.shape

(58, 368)

### (5) 상품명 넣기 ( db의 table과 상품명 일치시키기)

#### (a) 감귤 합치기

In [142]:
tmp = pdf[pdf['product']=='감귤']
tmp.mean()[1:]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


2021-09-07    6855.0
2021-09-08    6859.0
2021-09-09    6859.0
2021-09-10    6901.0
2021-09-11       NaN
               ...  
2022-09-02       NaN
2022-09-03       NaN
2022-09-04       NaN
2022-09-05       NaN
2022-09-06       NaN
Length: 365, dtype: float64

In [ ]:
### 합친거 pdf에 넣어주기
pdf.iloc[1,3:] = tmp.mean()[1:]
pdf

In [146]:
### 감귤 행 2개이므로, 1개행 버리기
pdf.drop(2, inplace=True)
pdf.reset_index(drop=True, inplace=True)
pdf

,product_id,product,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,1038,사과,사과후지(중),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1054,감귤,감귤노지(s과),6855.0,6859.0,6859.0,6901.0,NaN,NaN,6890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1050,건블루베리,건블루베리수입(중품),3220,3220,3200,3200,NaN,NaN,3200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1049,건포도,건포도수입(중품),1033,1045,1045,1033,NaN,NaN,1033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1042,단감,단감단감(중품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1047,레몬,레몬수입(중품),8786,8786,9072,9065,NaN,NaN,9065,...,NaN,9507,9507,9568,9568,9614,NaN,NaN,9652,9652
6,1051,망고,망고수입(상품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1043,바나나,바나나수입(상품),284,284,284,284,NaN,NaN,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
8,1039,배,배신고(중),NaN,NaN,30830,25405,NaN,NaN,30428,...,NaN,NaN,NaN,NaN,23230,22780,NaN,NaN,22480,22480
9,1040,복숭아,복숭아백도(상),24376,24563,NaN,NaN,NaN,NaN,NaN,...,NaN,22455,22406,22817,23353,23036,NaN,NaN,22575,22789


#### (b) 참다래 & 땅콩은 상품2개이므로, 따로 넣어주기

In [147]:
# '참다래국산(상품)'          -> '참다래(국산)'
# '참다래그린뉴질랜드(상품)'  -> '참다래(그린 뉴질랜드)'
# '땅콩국산'                  -> '땅콩(국산)', 
# '땅콩수입(중품)'           -> ' 땅콩(수입)' 
# '감귤노지(s과)'            ->  '감귤

goods_name_change = {'참다래국산(상품)' : '참다래(국산)', '참다래그린뉴질랜드(상품)' : '참다래(그린 뉴질랜드)' , '땅콩국산':'땅콩(국산)', '땅콩수입(중품)':'땅콩(수입)' ,'감귤노지(s과)' : '감귤'}
for i in range(len(pdf)):
    if pdf.loc[i,'goods_name'] in goods_name_change:
        pdf.loc[i,'goods_name'] = goods_name_change[pdf.loc[i,'goods_name']]

In [148]:
pdf   # 개수가 57개가 나와야 맞음  / 품목개수 55개 , 상품개수 57개

,product_id,product,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,1038,사과,사과후지(중),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1054,감귤,감귤,6855.0,6859.0,6859.0,6901.0,NaN,NaN,6890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1050,건블루베리,건블루베리수입(중품),3220,3220,3200,3200,NaN,NaN,3200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1049,건포도,건포도수입(중품),1033,1045,1045,1033,NaN,NaN,1033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1042,단감,단감단감(중품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1047,레몬,레몬수입(중품),8786,8786,9072,9065,NaN,NaN,9065,...,NaN,9507,9507,9568,9568,9614,NaN,NaN,9652,9652
6,1051,망고,망고수입(상품),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1043,바나나,바나나수입(상품),284,284,284,284,NaN,NaN,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
8,1039,배,배신고(중),NaN,NaN,30830,25405,NaN,NaN,30428,...,NaN,NaN,NaN,NaN,23230,22780,NaN,NaN,22480,22480
9,1040,복숭아,복숭아백도(상),24376,24563,NaN,NaN,NaN,NaN,NaN,...,NaN,22455,22406,22817,23353,23036,NaN,NaN,22575,22789


#### (c) 그 외 상품들 상품명 올바르게 교체하기

In [150]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/월별데이터(10년)

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/월별데이터(10년)


In [152]:
### id 정보 dictionary load
import pickle

# { 품목 : List[goods] }
with open('product_goods_dict.pickle', 'rb') as fr:
    product_goods_dict = pickle.load(fr)


# { 품목 : 품목id }
with open('product_productId_dict_nong.pickle','rb') as fr:
    product_productId_dict_nong = pickle.load(fr)

In [161]:
for i in range(len(pdf)):
    if len(product_goods_dict[pdf.loc[i,'product']])==1 :  #  품목당 상품이 1개만 있는 데이터 경우에 바꿔주기
        pdf.loc[i,'goods_name'] = product_goods_dict[pdf.loc[i,'product']]

In [162]:
pdf

,product_id,product,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,1038,사과,사과(후지),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1054,감귤,감귤,6855.0,6859.0,6859.0,6901.0,NaN,NaN,6890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1050,건블루베리,건블루베리(수입),3220,3220,3200,3200,NaN,NaN,3200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1049,건포도,건포도(수입),1033,1045,1045,1033,NaN,NaN,1033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1042,단감,단감,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1047,레몬,레몬(수입),8786,8786,9072,9065,NaN,NaN,9065,...,NaN,9507,9507,9568,9568,9614,NaN,NaN,9652,9652
6,1051,망고,망고(수입),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1043,바나나,수입,284,284,284,284,NaN,NaN,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
8,1039,배,배(신고),NaN,NaN,30830,25405,NaN,NaN,30428,...,NaN,NaN,NaN,NaN,23230,22780,NaN,NaN,22480,22480
9,1040,복숭아,백도,24376,24563,NaN,NaN,NaN,NaN,NaN,...,NaN,22455,22406,22817,23353,23036,NaN,NaN,22575,22789


### (7) 저장하기 (2차전처리)

In [163]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/대형마트/농산물유통정보

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/대형마트/농산물유통정보


In [165]:
pdf.to_excel('농산물유통정보_일데이터_대형마트_1년_상품_가격_2차전처리.xlsx',header=True, index=False)
pdf

,product_id,product,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,1038,사과,사과(후지),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1054,감귤,감귤,6855.0,6859.0,6859.0,6901.0,NaN,NaN,6890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1050,건블루베리,건블루베리(수입),3220,3220,3200,3200,NaN,NaN,3200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1049,건포도,건포도(수입),1033,1045,1045,1033,NaN,NaN,1033,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1042,단감,단감,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1047,레몬,레몬(수입),8786,8786,9072,9065,NaN,NaN,9065,...,NaN,9507,9507,9568,9568,9614,NaN,NaN,9652,9652
6,1051,망고,망고(수입),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1043,바나나,수입,284,284,284,284,NaN,NaN,284,...,NaN,300,300,300,300,313,NaN,NaN,298,298
8,1039,배,배(신고),NaN,NaN,30830,25405,NaN,NaN,30428,...,NaN,NaN,NaN,NaN,23230,22780,NaN,NaN,22480,22480
9,1040,복숭아,백도,24376,24563,NaN,NaN,NaN,NaN,NaN,...,NaN,22455,22406,22817,23353,23036,NaN,NaN,22575,22789


## [3] 3차 전처리 - goods table 만들기 & goods_goodsId Dictionary 만들기
+ goods 의 pk는 20000부터 시작

### (1) data 가져오기

In [168]:
cd /content/drive/MyDrive/Team_lemonmul/PopData/Data/대형마트/농산물유통정보

/content/drive/.shortcut-targets-by-id/1k2RzEvWtJ__WV4ThLWM5VyXk_FspqjfH/Team_lemonmul/PopData/Data/대형마트/농산물유통정보


In [180]:
import pandas as pd
pdf = pd.read_excel('농산물유통정보_일데이터_대형마트_1년_상품_가격_2차전처리.xlsx')
pdf

,product_id,product,goods_name,2021-09-07,2021-09-08,2021-09-09,2021-09-10,2021-09-11,2021-09-12,2021-09-13,...,2022-08-28,2022-08-29,2022-08-30,2022-08-31,2022-09-01,2022-09-02,2022-09-03,2022-09-04,2022-09-05,2022-09-06
0,1038,사과,사과(후지),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1054,감귤,감귤,6855.0,6859.0,6859.0,6901.0,NaN,NaN,6890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1050,건블루베리,건블루베리(수입),3220.0,3220.0,3200.0,3200.0,NaN,NaN,3200.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1049,건포도,건포도(수입),1033.0,1045.0,1045.0,1033.0,NaN,NaN,1033.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1042,단감,단감,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1047,레몬,레몬(수입),8786.0,8786.0,9072.0,9065.0,NaN,NaN,9065.0,...,NaN,9507.0,9507.0,9568.0,9568.0,9614.0,NaN,NaN,9652.0,9652.0
6,1051,망고,망고(수입),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1043,바나나,수입,284.0,284.0,284.0,284.0,NaN,NaN,284.0,...,NaN,300.0,300.0,300.0,300.0,313.0,NaN,NaN,298.0,298.0
8,1039,배,배(신고),NaN,NaN,30830.0,25405.0,NaN,NaN,30428.0,...,NaN,NaN,NaN,NaN,23230.0,22780.0,NaN,NaN,22480.0,22480.0
9,1040,복숭아,백도,24376.0,24563.0,NaN,NaN,NaN,NaN,NaN,...,NaN,22455.0,22406.0,22817.0,23353.0,23036.0,NaN,NaN,22575.0,22789.0


### (2) dataframe 만들기

In [179]:
import pandas as pd


goods_table = pd.DataFrame(columns=['goods_id','product_id','name','img','capacity','measure','ea','cheap_url']) # 상품id , 품목id, 상품명, 이미지, 개별용량, 용량명, 개수, 최저가url
goods_table

,goods_id,product_id,name,img,capacity,measure,ea,cheap_url


In [183]:
import numpy as np
goods_id = list(range(20000, 20000 + len(pdf)))
product_id =  list(pdf.loc[:,'product_id'])
name = list(pdf.loc[:,'goods_name'])
img = [f"{goods}_img" for goods in name]
capacity = [np.nan]*len(pdf)
measure = [np.nan]*len(pdf)
ea = [np.nan]*len(pdf)
cheap_url = [f"{goods}_url" for goods in name]

goods_table['goods_id']    = goods_id
goods_table['product_id']  = product_id
goods_table['name']        = name
goods_table['img']         = img
goods_table['capacity']    = capacity
goods_table['measure']     = measure
goods_table['ea']          = ea
goods_table['cheap_url']   = cheap_url

goods_table

,goods_id,product_id,name,img,capacity,measure,ea,cheap_url
0,20000,1038,사과(후지),사과(후지)_img,NaN,NaN,NaN,사과(후지)_url
1,20001,1054,감귤,감귤_img,NaN,NaN,NaN,감귤_url
2,20002,1050,건블루베리(수입),건블루베리(수입)_img,NaN,NaN,NaN,건블루베리(수입)_url
3,20003,1049,건포도(수입),건포도(수입)_img,NaN,NaN,NaN,건포도(수입)_url
4,20004,1042,단감,단감_img,NaN,NaN,NaN,단감_url
5,20005,1047,레몬(수입),레몬(수입)_img,NaN,NaN,NaN,레몬(수입)_url
6,20006,1051,망고(수입),망고(수입)_img,NaN,NaN,NaN,망고(수입)_url
7,20007,1043,수입,수입_img,NaN,NaN,NaN,수입_url
8,20008,1039,배(신고),배(신고)_img,NaN,NaN,NaN,배(신고)_url
9,20009,1040,백도,백도_img,NaN,NaN,NaN,백도_url


In [ ]:
import pandas as pd
import datetime

tmp = pd.read_excel('./과일류/농산물유통정보_대형마드_과일류_사과_후지(중).xlsx', header=None)
tmp.loc[0,2] + datetime.timedelta(days=35)
tmp

In [ ]:
veg_chilipowder = pd.read_excel('./채소류/농산물유통정보_대형마트_채소류_고춧가루_국산.xlsx')
veg_chilipowder

In [ ]:
fruit_citrus = pd.read_excel('./과일류/농산물유통정보_대형마트_과일류_감귤_노지(s과).xlsx')
fruit_citrus